<a href="https://colab.research.google.com/github/turgonboyev/Google_Colab_uchun/blob/main/Modelni_yaratish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd
import numpy as np
import sklearn

URL = 'https://github.com/ageron/handson-ml2/blob/master/datasets/housing/housing.csv?raw=true'

df = pd.read_csv(URL)

from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

housing = train_set.drop('median_house_value', axis=1)
housing_label = train_set['median_house_value'].copy()
housing_num = housing.drop('ocean_proximity', axis=1)

In [16]:
from sklearn.base import BaseEstimator, TransformerMixin
# bizga kerak ustunlar indekslari
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self # bizni funksiyamiz faqat transformer. estimator emas
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room: # add_bedrooms_per_room ustuni ixtiyoriy bo'ladi
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

In [21]:
#Pipeline yasaymiz

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

num_pipeline = Pipeline([
          ('imputer', SimpleImputer(strategy='median')),
          ('attribs_adder', CombinedAttributesAdder(add_bedrooms_per_room = True)),
          ('std_scaler', StandardScaler())             
])

num_pipeline.fit_transform(housing_num)

array([[ 1.27258656, -1.3728112 ,  0.34849025, ..., -0.17491646,
         0.05137609, -0.2117846 ],
       [ 0.70916212, -0.87669601,  1.61811813, ..., -0.40283542,
        -0.11736222,  0.34218528],
       [-0.44760309, -0.46014647, -1.95271028, ...,  0.08821601,
        -0.03227969, -0.66165785],
       ...,
       [ 0.59946887, -0.75500738,  0.58654547, ..., -0.60675918,
         0.02030568,  0.99951387],
       [-1.18553953,  0.90651045, -1.07984112, ...,  0.40217517,
         0.00707608, -0.79086209],
       [-1.41489815,  0.99543676,  1.85617335, ..., -0.85144571,
        -0.08535429,  1.69520292]])

In [20]:
from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = ['ocean_proximity']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs)
])

housing_prepared = full_pipeline.fit_transform(housing)

In [23]:
from sklearn.linear_model import LinearRegression

LR_model = LinearRegression()

LR_model.fit(housing_prepared, housing_label)

LinearRegression()

In [25]:
test_data = housing.sample(10)
test_label = housing_label.loc[test_data.index]

test_data_prepared = full_pipeline.transform(test_data)

predicted_labels = LR_model.predict(test_data_prepared)

pd.DataFrame({'Bashorat': predicted_labels, 'Real narx': test_label})

,Bashorat,Real narx
5846,214996.602587,200000.0
10189,257107.614237,175800.0
11740,114546.075549,216500.0
9327,320681.186933,500001.0
13743,163396.787816,135800.0
17374,178354.711601,136900.0
9409,336117.333107,400400.0
6511,129126.319235,184400.0
3862,241419.386498,337500.0
249,178946.640278,131900.0
